In [46]:
import rpy2.robjects as robjects
from rpy2.robjects import numpy2ri
import rpy2
from pprint import pprint
import re

In [44]:
path = "../../david_eccles_bc_ideas/"
SRATE = 5000

In [2]:
robjects.r['load']("../../david_eccles_bc_ideas/mtDNA_mappable_called_with_rawSignal.Rdata")

<StrVector - Python:0x7fc95dba4a88 / R:0x3901e48>
['nanopore.bundled']

In [105]:
data = numpy2ri.ri2py(robjects.r['nanopore.bundled'])

In [6]:
files = !ls ../../david_eccles_bc_ideas/processed/ | grep fast5

In [89]:
n = 0
file = files[n]

In [90]:
result = re.search(r'_ch(\d+)_file(\d+)_', file)
file_id = result.group(2)
channel_id = result.group(1)

only look at template strand for the beginning

In [37]:
seq = !poretools fasta {path+"processed/" + file} | grep template -A1 | grep -v ">"

In [39]:
events = !poretools events {path+"processed/" + file} | grep template

file	strand	mean	start	stdv	length	model_state	model_level	move	p_model_state	mp_model_state	p_mp_model_state	p_A	p_C	p_G	p_T	raw_index

In [42]:
events = [line.split("\t") for line in events]

1 = reference
2 = called
3 = channel
8 = metrichor results (events)
9 = "mux"
10 = signal range 
11 = signal

In [94]:
raw = data[11][n]

kindly, david provides us with the exact reference for the reads

In [136]:
refs = dict()
for key, seq in zip(data[1].names, data[1]):
    res = re.search(r'channel_(\d+)/(.*)file(\d+)_', key)
    chid = res.group(1)
    fid = res.group(3)
    try: 
        refs[chid]
    except KeyError:
        refs[chid] = dict()
    refs[chid][fid] = seq

{'132': {'10': 'CTACGGGATTGGGGGCTAACCGTGCCATACGCATTCTTCTATTAAAACTAGCTACTCATCGCTCTTCTCAATCATCTTAATTTATATCAGTCTCAGGAAGTGCGCGACACCACTCTAAAGTCTATATCCGTGTGGTAGTATAAACTCATTACTCTGGTCTTAACCTCCGTAGGAGTGTCTCTTCCCCACCATCGAAGAAGGAGCTATACGCACCACCAGCCCAAAAGCTGACTTCAGTTGATATAATTCTTGAGTACATAGTTACATAGTACAACAATACATTTAATGACTGCGTCATGCATAACCTTTCCCCAAGCAGCTAGTACATGCGTAATGGTTCATATCCTCATACGTACTGCTCATAAATCAATATATACCATGAAATATTATCTGCTCACCATTAAACAGTGTTACACATATCTGTTATCCTCCACCATACAGTCACTATACTTCTCTTCCCTGACTATCCCCTTCCCCATTTGGTCCTTAATCACATCCTCCAATTTGCATGAACAACAAAGATTAGGGTCTCATGAACTAAGTAAACTTAGGACTAATGAAACTTTAGAGCTATGGTTACTTCTAGGGCTTAAATGCGTTATCGCCGGCGTTCCCTTAAATGAGGCTCTCGATGGTATCGACATCAATCAGCCCATACAAGCCTAACTTGGTGTCATGCGTTTATTATCTTTTTGGTTTTGGCCTACCTTTCATCAACCTAGCCGTTAGGCAGGCAGCGCGCCCGTCAGGGCACCGTGGAATCATTAGTCCCAAAACATCCCAATCACTAAAAGGCTAATTATTCATTAAGCTTGTCCCTAGCTATAATGCTACTCAAATACCAATTTAACTCTCGACCCCGACACAACATATCATGCCAAACCAGAACACTAGAACTTGAAAAAGCTCTTAACTATCATCTAGGTGTCCTCTTGATTCCTGGTAATTCCCAAAGACCTTATTTTAATGGTCCACTGTTACAGG